# Hosting 50+ XGBoost models in a multi-model endpoint for millisecond latency
This example demonstrate hosting 51 State-wise ML models in a SageMaker Multi-Model Endpoint to predict customer churn based on account usage. The models are trained using a synthetic telecommunication customer churn dataset and SageMaker's built-in XGBoost algorithm. We will host this multi-model endpoint on two instance types: `ml.c5.xlarge` and `ml.c5.2xlarge` and compare the performance with a load test in order to find out an optimal hosting architecture. We will analyze the load testing results in Amazon CloudWatch.

Instead of hosting 51 models in 51 endpoints as illustrated below,

<img src='images/sagemaker-model-deployment-multi-endpoints.png' width='700'/>
<!-- ![mult-endpoints](images/sagemaker-model-deployment-multi-endpoints.png) -->

We can host 51 models in one endpoint and load models dynamically from S3.

<img src='images/sagemaker-model-deployment-multimodel.png' width='700'/>
<!-- ![mult-model-1](images/sagemaker-model-deployment-multimodel.png) -->

Amazon CloudWatch dashboard to show endpoint performance.
<!-- <img src='images/mme-load-testing-combined-1.png' width='900'/> -->
![cw-dashboard](images/mme-load-testing-combined-2.png)

This notebook is developed using `Python 3 (Data Science)` kernel with a ml.t3.medium instance.

First we install a library `sagemaker-experiment` to manage the training jobs.

In [ ]:
!pip install -q sagemaker-experiments

Import the libraries and set up the SageMaker resources.

In [ ]:
import sagemaker
import json
import boto3
import numpy as np
import pandas as pd

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker/reinvent21-aim408/churn-mme'

The dataset is a customer churn dataset from a synthetic telecommunication use case. We load the data directly from source.

In [ ]:
df=pd.read_csv('s3://sagemaker-sample-files/datasets/tabular/synthetic/churn.txt')
df['CustomerID']=df.index
df.head()

We perform minimal data preprocessing: 
1. replacing binary columns from string type to integers (0 & 1).
2. setting CustomerID as the dataframe index and move the target column to the first column for XGBoost training.

In [ ]:
binary_columns=["Int'l Plan", "VMail Plan"]
df[binary_columns] = df[binary_columns].replace(to_replace=['yes', 'no'], 
                                                value=[1, 0])

In [ ]:
df['Churn?'] = df['Churn?'].replace(to_replace=['True.', 'False.'], 
                                    value=[1, 0])

In [ ]:
columns=['Churn?', 'State', 'Account Length', "Int'l Plan",
         'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge',
         'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls',
         'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge',
         'CustServ Calls']
df.index = df['CustomerID']
df_processed = df[columns]

The processed data shown below.

In [ ]:
df_processed.head()

We hold out 10% of data as a test set, stratified by `State`. The remaining data will be further split into train and validation set later right before training.

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_processed, test_size=0.1, random_state=42, 
                                     shuffle=True, stratify=df_processed['State'])

Save the test data into S3 bucket. Two version of the test data are saved, one that has complete data, and the other one without target and index for inference purposes. 

In [ ]:
columns_no_target=['Account Length', "Int'l Plan", 'VMail Plan', 'VMail Message', 
                   'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 
                   'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 
                   'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls']

df_test.to_csv(f's3://{bucket}/{prefix}/churn_data/churn_test.csv')
df_test[columns_no_target].to_csv(f's3://{bucket}/{prefix}/churn_data/churn_test_no_target.csv', 
                                  index=False)

We set up an experiment in SageMaker to hold all the training job information.

In [ ]:
from sagemaker.amazon.amazon_estimator import image_uris
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
import time
from time import gmtime, strftime

dict_estimator = {}

experiment_name = 'churn-prediction'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Training churn prediction models based on telco churn dataset.')
except ClientError as e:
    experiment = Experiment.load(experiment_name)
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')

For convenience, we create a function `launch_training_job()` so that later we can reuse it in a loop through the States. The training algorithm used here is SageMaker's built-in XGBoost algorithm with 20 rounds of training as the only hyperparameter we specify. 

In [ ]:
image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')
train_instance_type = 'ml.m5.xlarge'
train_instance_count = 1
s3_output = f's3://{bucket}/{prefix}/churn_data/training'

def launch_training_job(state, train_data_s3, val_data_s3):
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'churn-xgb-{state}-{exp_datetime}'

    # Creating a new trial for the experiment
    exp_trial = Trial.create(experiment_name=experiment_name, 
                             trial_name=jobname)

    experiment_config={'ExperimentName': experiment_name,
                       'TrialName': exp_trial.trial_name,
                       'TrialComponentDisplayName': 'Training'}

    xgb = sagemaker.estimator.Estimator(image,
                                        role,
                                        instance_count=train_instance_count,
                                        instance_type=train_instance_type,
                                        output_path=s3_output,
                                        enable_sagemaker_metrics=True,
                                        sagemaker_session=sess)
    xgb.set_hyperparameters(objective='binary:logistic', 
                            num_round=20)
    
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_data_s3, 
                                                 content_type='csv')
    val_input = sagemaker.inputs.TrainingInput(s3_data=val_data_s3, 
                                               content_type='csv')
    data_channels={'train': train_input, 'validation': val_input}
    
    xgb.fit(inputs=data_channels, 
            job_name=jobname, 
            experiment_config=experiment_config, 
            wait=False)

    return xgb

We isolate the data points by `State`, create train and validation sets for each `State` and train models by `State` using `launch_training_job()`. Again we hold out 10% as validation set in each `State`. We save the estimators in a dictionary `dict_estimator`. 

Execute the next four cells to launch the training jobs if this is the first time running the demo. If you already have run the training jobs from this notebook and have completed trials in SageMaker Experiments, you can proceed to [loading the existing estimators](#loading-estimators).

In [ ]:
def wait_for_training_quota(dict_estimator, job_limit = 10, wait = 30):
    def query_jobs(dict_estimator):
        counter=0
        for key, estimator in dict_estimator.items():
            status = estimator.latest_training_job.describe()["TrainingJobStatus"]
            time.sleep(2)
            if status == "InProgress":
                counter+=1
        return counter
    
    job_count = query_jobs(dict_estimator)
    if job_count < job_limit:
        print(f'Current total running jobs {job_count} is below {job_limit}. Proceeding...')
        return 
    
    while job_count >= job_limit:
        print(f'Current total running jobs {job_count} is reaching the limit {job_limit}. Waiting {wait} seconds...')
        time.sleep(wait)
        job_count = query_jobs(dict_estimator)

    print(f'Current total running jobs {job_count} is below {job_limit}. Proceeding...')

In [ ]:
for state in df_processed.State.unique():
    print(state)
    out_train_csv_s3 = f's3://{bucket}/{prefix}/churn_data/by_state/churn_{state}_train.csv'
    out_val_csv_s3 = f's3://{bucket}/{prefix}/churn_data/by_state/churn_{state}_val.csv'
    
    # create train/val split for each State
    df_state = df_train[df_train['State']==state].drop(labels='State', axis=1)
    df_state_train, df_state_val = train_test_split(df_state, 
                                                    test_size=0.1, 
                                                    random_state=42, 
                                                    shuffle=True, 
                                                    stratify=df_state['Churn?'])
    
    df_state_train.to_csv(out_train_csv_s3, index=False)
    df_state_val.to_csv(out_val_csv_s3, index=False)
    
    wait_for_training_quota(dict_estimator, job_limit=10, wait=30)
    
    dict_estimator[state] = launch_training_job(state, out_train_csv_s3, out_val_csv_s3)
    
    time.sleep(2)

Wait for all jobs to complete.

In [ ]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f"Waiting for job: {job}")
    status = estimator.latest_training_job.describe()["TrainingJobStatus"]
    while status == "InProgress":
        time.sleep(45)
        status = estimator.latest_training_job.describe()["TrainingJobStatus"]
        if status == "InProgress":
            print(f"{job} job status: {status}")
    print(f"DONE. Status for {job} is {status}\n")

In [ ]:
for est in list(dict_estimator.values()):
    wait_for_training_job_to_complete(est)

<a name="loading-estimators"></a>The code snippet below is to retrieve the estimators from the experiment trials. It is useful when you have already trained the models but somehow lost the dictionary `dict_estimator` and want to resume the work.

```python
dict_estimator={}
experiment = Experiment.load(experiment_name)
for i, j in enumerate(experiment.list_trials()):
    print(i, j.trial_name)
    jobname=j.trial_name
    state=jobname.split('-')[2]
    print(state)
    try:
        dict_estimator[state]=sagemaker.estimator.Estimator.attach(jobname)
    except:
        pass
```

In [ ]:
## Uncomment this part to load the estimators if you already have trained them.
# dict_estimator={}
# experiment = Experiment.load(experiment_name)
# for i, j in enumerate(experiment.list_trials()):
#     print(i, j.trial_name)
#     jobname=j.trial_name
#     state=jobname.split('-')[2]
#     print(state)
#     try:
#         dict_estimator[state]=sagemaker.estimator.Estimator.attach(jobname)
#     except:
#         pass

Once the training are completed, we can start hosting our multimodel endpoint. We host our State-wise multi-model endpoint in two different instances: `ml.c5.xlarge` and `ml.c5.2xlarge`. And we will be conducting load testing to profile the performance.

In [ ]:
print(len(dict_estimator))

In [ ]:
print(dict_estimator.keys())

Here we designate a S3 location to hold all the model artifacts we would like to host. At any time (before or after the endpoint is created), we can dynamically add models to the designated model artifacts folder, making multi-model endpoint a flexible tool to serve models at scale.

In [ ]:
model_data_prefix = f's3://{bucket}/{prefix}/churn_data/multi_model_artifacts'

In [ ]:
for state, est in dict_estimator.items():
    artifact_path = est.model_data
    state_model_name = f'churn-xgb-{state}.tar.gz'
    print(f'Copying {state_model_name} to multi_model_artifacts folder')
    # This is copying over the model artifact to the S3 location for the MME.
    !aws s3 --quiet cp {artifact_path} {model_data_prefix}/{state_model_name}

Endpoint creation is a three-step process with the API. `create_model()`==>`create_endpoint_config()`==>`creat_endpoint()`.

Create our first endpoint with `ml.c5.xlarge` instance which has 4 vCPU and 8 GB RAM.

In [ ]:
exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
model_name = f'churn-xgb-mme-{exp_datetime}'

hosting_instance_type = 'ml.c5.xlarge'
hosting_instance_count = 1

endpoint_name = f'{model_name}-c5-xl'

# image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')
container = {'Image': image, 
             'ModelDataUrl': model_data_prefix, 
             'Mode': 'MultiModel'}

response1 = sess.sagemaker_client.create_model(ModelName = model_name,
                                               ExecutionRoleArn = role,
                                               Containers = [container])

In [ ]:
response2 = sess.sagemaker_client.create_endpoint_config(
                EndpointConfigName = endpoint_name,
                ProductionVariants = [{'InstanceType': hosting_instance_type,
                                       'InitialInstanceCount': hosting_instance_count,
                                       'InitialVariantWeight': 1,
                                       'ModelName': model_name,
                                       'VariantName': 'AllTraffic'}])

In [ ]:
response3 = sess.sagemaker_client.create_endpoint(EndpointName = endpoint_name,
                                                  EndpointConfigName = endpoint_name)
print(endpoint_name)

We create another endpoint with `ml.c5.2xlarge` which has 8 vCPU and 16 GB RAM.

In [ ]:
hosting_instance_type = 'ml.c5.2xlarge'
hosting_instance_count = 1

endpoint_name_2 = f'{model_name}-c5-2xl'

response4 = sess.sagemaker_client.create_endpoint_config(
                EndpointConfigName = endpoint_name_2,
                ProductionVariants = [{'InstanceType': hosting_instance_type,
                                       'InitialInstanceCount': hosting_instance_count,
                                       'InitialVariantWeight': 1,
                                       'ModelName': model_name, # re-using the model
                                       'VariantName': 'AllTraffic'}])

In [ ]:
response5 = sess.sagemaker_client.create_endpoint(EndpointName = endpoint_name_2,
                                                  EndpointConfigName = endpoint_name_2)
print(endpoint_name_2)

In [ ]:
waiter = sess.sagemaker_client.get_waiter('endpoint_in_service')
print(f'Waiting for endpoint {endpoint_name} to create...')
waiter.wait(EndpointName=endpoint_name)
print(f'Waiting for endpoint {endpoint_name_2} to create...')
waiter.wait(EndpointName=endpoint_name_2)

Let's move our load testing to [AWS Cloud9](https://console.aws.amazon.com/cloud9/home?region=us-east-1). You could also use your local computer to run the load testing.

### (Optional) Enable autoscaling
We have verified the baseline single instance performance, let's apply a autoscaling policy to allow scale in/out between 2 to 5 instances for variable traffic to ensure performance. Here we use a predefined metric `SageMakerVariantInvocationsPerInstance` with a `TargetValue` 4,000 to balance the load to 4,000 requests per instance.

In [ ]:
# Common class representing Application Auto Scaling for SageMaker amongst other services
client = boto3.client('application-autoscaling')

# This is the format in which application autoscaling references the endpoint
resource_id=f'endpoint/{endpoint_name_2}/variant/AllTraffic' 

response = client.register_scalable_target(
    ServiceNamespace='sagemaker', 
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=2,  
    MaxCapacity=5
)

response = client.put_scaling_policy(
    PolicyName='Invocations-ScalingPolicy',
    ServiceNamespace='sagemaker', # The namespace of the AWS service that provides the resource. 
    ResourceId=resource_id, # Endpoint name 
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', # SageMaker supports only Instance Count
    PolicyType='TargetTrackingScaling', # 'StepScaling'|'TargetTrackingScaling'
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 4000, # The target value for the metric: ApproximateBacklogSizePerInstance. 
        'PredefinedMetricSpecification': {
            'PredefinedMetricType': 'SageMakerVariantInvocationsPerInstance', 
        },
        'ScaleInCooldown': 600, # The cooldown period helps you prevent your Auto Scaling group from launching or terminating 
                                # additional instances before the effects of previous activities are visible. 
                                # You can configure the length of time based on your instance startup time or other application needs.
                                # ScaleInCooldown - The amount of time, in seconds, after a scale in activity completes before another scale in activity can start. 
        'ScaleOutCooldown': 300,# ScaleOutCooldown - The amount of time, in seconds, after a scale out activity completes before another scale out activity can start.
        
        'DisableScaleIn': False,# Indicates whether scale in by the target tracking policy is disabled. 
                                # If the value is true , scale in is disabled and the target tracking policy won't 
                                # remove capacity from the scalable resource.
    }
)

After you are done with the load-testing, uncomment and run the next cell to delete endpoints to stop incurring cost.

In [ ]:
# sess.sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
# sess.sagemaker_client.delete_endpoint(EndpointName=endpoint_name_2)